# 水平集函数初始化为符号距离函数
  1. 定义孔洞
圆形孔洞的界面定义为圆的边界，即 $(x - cx)^2 + (y - cy)^2 = cr^2$，其中 $(cx,cy)$ 是圆心，$cr$ 是半径。

  1. 计算距离
计算每个网格点带所有孔洞圆心的欧式距离
$$d_i = \sqrt{(x-cx_i)^2 + (y-cy_i)^2} - cr$$
其中 $i$ 表示第 $i$ 个孔洞。

  3. 构建水平集函数
  $$\Phi(x,y) = \min_id_i(x,y)$$
  其中 $\Phi(x,y)$ 表示水平集网格点 $(x,y)$ 的水平集函数值，其中 $\Phi$ 满足
$$
\begin{cases}
\Phi(x,y) = 0 &在圆边界上\\
\Phi(x,y) > 0 &在圆外\\
\Phi(x,y) < 0 &在圆内
\end{cases}$$

# 求解水平集重置化方程
    
1. 时间步长取为：

$$\Delta{t} = \frac{1}{2}\frac{\Delta_{\min}}{\max~\big|(\mathrm{sign}(\Phi_n)_{ij}\big|},\quad\Delta_{\min} = \min(\Delta{x},\Delta{y})$$
    
2. 数值计算时可以通过磨光：
$$\mathrm{sign}(\Phi)=\frac{\Phi}{\sqrt{\Phi^2+|\nabla\Phi|^2(\Delta x)^2}}$$
其中 $\nabla\Phi$ 采用中心差分近似，$D_{ij}^{cx}$ 和 $D_{ij}^{cy}$ 分别是两个维度中的中心差分算子。

3. 计算格式：
$$\Phi_{ij}^{n+1} = \Phi_{ij}^n - \Delta{t}\bigg(\max\big(\mathrm{sign}(\Phi_{ij}^n),0\big)\nabla^{+} + \min\big(\mathrm{sign}(\Phi_{ij}^n),0\big)\nabla^{-} - \mathrm{sign}(\Phi_{ij}^n)\bigg)$$
其中
$$\nabla^{+} = \big[\max(D_{ij}^{-x},0)^2 + \min(D_{ij}^{+x},0)^2 + \max(D_{ij}^{-y},0)^2 + \min(D_{ij}^{+y},0)^2\big]^{1/2}$$
$$\nabla^{-} = \big[\max(D_{ij}^{+x},0)^2 + \min(D_{ij}^{-x},0)^2 + \max(D_{ij}^{+y},0)^2 + \min(D_{ij}^{-y},0)^2\big]^{1/2}$$
这里，$\Delta{t}$ 是时间步长，$D_{ij}^{\pm{x}}$ 和 $D_{ij}^{\pm{y}}$ 分别是 $\bm{x}\in\mathbb{R}^2$ 的两个维度中的向前和向后差分算子。

In [ ]:
import numpy as np

def upwind_diff(phi, d, direction):
    """
    使用迎风格式计算向前和向后有限差分.

    Parameters:
    - phi ( ndarray - (nely+2, nlex+2) ): 水平集函数在水平集网格点上的值;
    - d (float): x 或 y 方向上相邻网格的间距, 取决于 'direction' 参数;
    - direction (str): 'x': 沿 x 方向计算差分, 'y': 表示沿 y 方向计算差分.

    Returns:
    - back_diff ( ndarray - (nely+2, nlex+2) ): 向后差分矩阵: ( phi(i, j) - phi(i-1, j) ) / dx
                            或 ( phi(i, j) - phi(i, j-1) ) / dx, 取决于 'direction'.
    - fawd_diff ( ndarray - (nely+2, nlex+2) ): 向前差分矩阵: ( phi(i+1, j) - phi(i, j) ) / dx
                            或 ( phi(i, j+1) - phi(i, j) ) / dx, 取决于 'direction'.
    """
    # 根据指定的方向计算后向和前向差分
    if direction == 'x':
        x_minus_1 = np.roll(phi, 1, axis=1) # x 方向向右位移
        x_plus_1 = np.roll(phi, -1, axis=1) # x 方向向左位移
        dx = d
        back_diff = (phi - x_minus_1) / dx
        fawd_diff = (x_plus_1 - phi) / dx
    elif direction == 'y':
        y_minus_1 = np.roll(phi, 1, axis=0) # y 方向向下位移
        y_plus_1 = np.roll(phi, -1, axis=0) # y 方向向上位移
        dy = d
        back_diff = (phi - y_minus_1) / dy
        fawd_diff = (y_plus_1 - phi) / dy
    else:
        raise ValueError("direction 必须是 'x' 或 'y'")
    
    return back_diff, fawd_diff

def reinit(phi0, dx, dy, loop_num):
    """ 
    将水平集函数重初始化为符号距离函数.

    Parameters:
    - phi0 ( ndarray - (nely+2, nelx+2) ): 重置化前的水平集界面;
    - dx: 有限元单元的宽度;
    - dy: 有限元单元的高度;
    - loop_num: 重初始化的时间步数.

    Returns:
    - sign_dist_phi (ndarray - (ls_NN, )): 重置化后的水平集界面.
    """
    for _ in range(loop_num + 1):
        # 水平集函数沿 x 和 y 方向的向前和向后差分算子
        dx_L, dx_R = upwind_diff(phi0, dx, 'x')
        dy_L, dy_R = upwind_diff(phi0, dy, 'y')
        
        # 水平集函数沿 x 和 y 方向的中心差分算子
        dx_C = (dx_L + dx_R) / 2
        dy_C = (dy_L + dy_R) / 2
        
        # sign(Phi) 的数值计算
        signPhi = phi0 / (np.sqrt(phi0**2 + (dx_C**2 + dy_C**2) * dx**2) + np.finfo(float).eps)

        # CFL 时间步长
        detT = 0.5 * min(dx, dy) / np.max(np.abs(signPhi))
        
        grad_plus = np.sqrt(np.maximum(dx_L, 0)**2 + np.minimum(dx_R, 0)**2 +
                            np.maximum(dy_L, 0)**2 + np.minimum(dy_R, 0)**2)
        grad_minus = np.sqrt(np.minimum(dx_L, 0)**2 + np.maximum(dx_R, 0)**2 +
                            np.minimum(dy_L, 0)**2 + np.maximum(dy_R, 0)**2)

        phi0 = phi0 - detT * \
            (np.maximum(signPhi, 0)*grad_plus + np.minimum(signPhi, 0)*grad_minus - signPhi)

    sign_dist_phi = phi0.flatten('F')

    return sign_dist_phi

# 检验 reinit 函数

In [6]:
from fealpy.mesh import UniformMesh2d

# 测试参数
nelx, nely = 32, 20
domain = [0, 32, 0, 20]
dw = domain[1] - domain[0]
dh = domain[3] - domain[2]
ew = (domain[1]-domain[0]) / nelx 
eh = (domain[3]-domain[2]) / nely 

from fealpy.mesh import UniformMesh2d
ls_domain = [-ew/2, dw+ew/2, -eh/2, dh+eh/2]
ls_mesh = UniformMesh2d(extent=(0, nelx+1, 0, nely+1), 
                    h=(ew, eh), origin=(ls_domain[0], ls_domain[2]))
ls_NC = ls_mesh.number_of_cells()
print("ls_NC:", ls_NC)
ls_NN = ls_mesh.number_of_nodes()
print("ls_NN:", ls_NN)
ls_node = ls_mesh.entity('node')
print("ls_node:", ls_node.shape, "\n", ls_node)

ls_NC: 693
ls_NN: 748
ls_node: (748, 2) 
 [[-0.5 -0.5]
 [-0.5  0.5]
 [-0.5  1.5]
 ...
 [32.5 18.5]
 [32.5 19.5]
 [32.5 20.5]]


In [13]:
import numpy as np
from fealpy.decorator import cartesian

# 非符号距离函数的水平集函数
@cartesian
def phi_nsd(p):
    x = p[..., 0]
    y = p[..., 1]
    val = (x - 0.5)**2 + (y - 0.75)**2 - 0.0115

    return val

# 符号距离函数的水平集函数
@cartesian
def phi_sd(p):
    x = p[..., 0]
    y = p[..., 1]
    val = np.sqrt((x - 0.5)**2 + (y - 0.75)**2) - 0.15

    return val

lsf_nsd = phi_nsd(ls_node)
print("lsf_nsd:", lsf_nsd.shape, "\n", lsf_nsd)
lsf_sd = phi_sd(ls_node)
print("lsf_sd:", lsf_sd.shape, "\n", lsf_sd)

lsf_nsd: (748,) 
 [2.551000e+00 1.051000e+00 1.551000e+00 4.051000e+00 8.551000e+00
 1.505100e+01 2.355100e+01 3.405100e+01 4.655100e+01 6.105100e+01
 7.755100e+01 9.605100e+01 1.165510e+02 1.390510e+02 1.635510e+02
 1.900510e+02 2.185510e+02 2.490510e+02 2.815510e+02 3.160510e+02
 3.525510e+02 3.910510e+02 1.551000e+00 5.100000e-02 5.510000e-01
 3.051000e+00 7.551000e+00 1.405100e+01 2.255100e+01 3.305100e+01
 4.555100e+01 6.005100e+01 7.655100e+01 9.505100e+01 1.155510e+02
 1.380510e+02 1.625510e+02 1.890510e+02 2.175510e+02 2.480510e+02
 2.805510e+02 3.150510e+02 3.515510e+02 3.900510e+02 2.551000e+00
 1.051000e+00 1.551000e+00 4.051000e+00 8.551000e+00 1.505100e+01
 2.355100e+01 3.405100e+01 4.655100e+01 6.105100e+01 7.755100e+01
 9.605100e+01 1.165510e+02 1.390510e+02 1.635510e+02 1.900510e+02
 2.185510e+02 2.490510e+02 2.815510e+02 3.160510e+02 3.525510e+02
 3.910510e+02 5.551000e+00 4.051000e+00 4.551000e+00 7.051000e+00
 1.155100e+01 1.805100e+01 2.655100e+01 3.705100e+01 4.955

In [15]:
import os
visualization_dir = 'visualization/'
os.makedirs(visualization_dir, exist_ok=True)
fname = os.path.join(visualization_dir, 'wang_reinit.vts')
ls_mesh.to_vtk(filename=fname, 
               nodedata={'lsf_nsd': lsf_nsd,
                         'lsf_sd': lsf_sd})

'visualization/wang_reinit.vts'